<a href="https://colab.research.google.com/github/AshvinVignesh/DL_CW_2/blob/main/Bert_Torch_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.0 MB/s eta 0:00:00


In [2]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__


('4.35.2', '0.24.1')

In [3]:
import pandas as pd

In [6]:
cleaned = pd.read_csv("yelp_reviews_2500.csv")
cleaned

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,BCTicX3VmcbB87zMM9_RjQ,x4f9anzaaBozvJh5nv26SA,z2FRDJSxRR3mmC9syR25xA,3.0,0,0,0,Cool structure to go see if visiting Nashville...,2010-03-20 01:48:14
1,325oPp4B8yzEPEQ63pv3fA,GiFvlh2yiap62XTb32cf9A,6_T2xzR74JqGCTPefAD8Tw,3.0,0,0,0,We planned a trip to Philly around my husbands...,2021-07-02 13:11:42
2,HrqfrJ3wET8z60G5EanYQw,ykN3-oxN747QMe96097KeA,Cp-xqpD0qY97LziqEUdJtw,3.0,3,0,0,Always good service. Room was clean except the...,2018-07-18 09:15:19
3,o6Wz0sgEVOtbMI-ZHwW5eA,RdJ7Ykelg8w4y5FTmxjT8g,bdgRENPF17SYmEn2k5VPQA,3.0,7,0,0,"This place has been here for years and years, ...",2007-08-09 01:09:15
4,AnDA5ykl8O6bpnstvVRgUw,Mq3weWcIs6tTwf4QGGkmrQ,yNlR9wTzd6Hg7It9fQrhdg,3.0,0,0,0,This is a very large location (with a ton of T...,2018-04-17 12:32:43
...,...,...,...,...,...,...,...,...,...
2495,KEyzMHNIexmuATBTp1yR_Q,NMLvjdY7IOdtfU0TepvUuA,6LztYkV0YY0q9Ky93JKQVQ,2.0,1,0,0,10/2/16 Update. I have been back twice since ...,2016-10-02 18:49:10
2496,kt-IqmBd43IIqUFNzVZguA,bvE4RfLMcsa8XAt3a2R5ag,alV1GLapqmyr1Ef51qKrfg,2.0,4,0,0,We were excited about Certa Pro completing our...,2017-08-26 15:41:19
2497,gxsGaKiOAgAVVAQe76hBLQ,LOtETDlJqaB4EB2UoGW9mw,yEmEDomLUofVtbZoUeFV5g,2.0,0,0,0,Driven by numerous times and finally came in. ...,2016-01-25 18:54:16
2498,Nx434DAAvU3RbO-uQgnJkw,ik5XVZy4vhKyInFTp542Wg,duNA78geUZkvfagznBPjkQ,2.0,0,0,0,Let me say that the service was excellent. The...,2010-01-13 07:04:30


In [7]:
cleaned=cleaned.drop(cleaned.columns[[0,1,2,4,5,6,8]], axis = 1)
cleaned

,stars,text
0,3.0,Cool structure to go see if visiting Nashville...
1,3.0,We planned a trip to Philly around my husbands...
2,3.0,Always good service. Room was clean except the...
3,3.0,"This place has been here for years and years, ..."
4,3.0,This is a very large location (with a ton of T...
...,...,...
2495,2.0,10/2/16 Update. I have been back twice since ...
2496,2.0,We were excited about Certa Pro completing our...
2497,2.0,Driven by numerous times and finally came in. ...
2498,2.0,Let me say that the service was excellent. The...


In [8]:
cleaned['stars'].value_counts()

3.0    500
5.0    500
4.0    500
1.0    500
2.0    500
Name: stars, dtype: int64

In [9]:
def map_sentiment_scores(star_number):
    star_number = int(star_number)
    if star_number == 1:
        return 0
    elif star_number == 2:
        return 1
    elif star_number == 3:
        return 2
    elif star_number == 4:
        return 3
    elif star_number == 5:
        return 4

In [10]:
cleaned['stars'] = cleaned.stars.apply(map_sentiment_scores)
cleaned.head(5)

,stars,text
0,2,Cool structure to go see if visiting Nashville...
1,2,We planned a trip to Philly around my husbands...
2,2,Always good service. Room was clean except the...
3,2,"This place has been here for years and years, ..."
4,2,This is a very large location (with a ton of T...


In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [12]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=5)

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
model = model.to('cuda')

In [14]:
sample_data = ["I am eating","I am playing "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [15]:
X = list(cleaned['text'])
y = list(cleaned['stars'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [16]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [17]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [18]:
len(X_train),len(X_val)

(2000, 500)

In [19]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [20]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [21]:
train_dataset[5]

{'input_ids': tensor([  101,  1996,  3587,  5217,  2003,  1037,  4825,  2011,  2358,  1012,
          3434,  2717, 21159,  3686,  3126,  2585,  8925,  1006,  1997,  8925,
          1005,  1055,  2846,  1998,  8925,  1005,  1055,  7967,  3347,  1007,
          2284,  2503,  1996,  1018, 11774,  2015, 12161,  1012,  2017,  2344,
          2012,  1996,  3347,  2006,  1996,  2364,  2723,  1998,  1996,  2833,
          2003,  5359,  2000,  2115,  2795,  1006,  2045,  2003, 10747,  2006,
          1996,  2364,  2504,  1010,  2006,  1996,  4397,  2441,  8721,  1010,
          1998,  2130,  2648,  2006,  1996, 19404,  1007,  1012,  2026,  2177,
          3641,  1996, 14397, 15006,  1006,  1002,  2184,  1007,  1011,  4840,
          2160, 21565, 11772,  1010,  2160, 21565, 11611,  1010,  2630,  8808,
          1010,  2665, 24444,  1010, 22861,  4160, 12901,  1010, 14768,  6949,
          1012, 24783,  4840,   999,  2025,  2460,  2006,  8808, 13675, 26607,
          2094,  2030,  2060, 22286,  2

In [22]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred,average='micro')
    precision = precision_score(y_true=labels, y_pred=pred,average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred,average='micro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [23]:

# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps = 100,
    eval_steps = 100,
    learning_rate = 2e-4,
    evaluation_strategy="steps",


)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


In [24]:
import matplotlib.pyplot as plt

In [25]:
train = trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,No log,1.671752,0.200000,0.200000,0.200000,0.200000
200,No log,1.633170,0.200000,0.200000,0.200000,0.200000
300,No log,1.633177,0.200000,0.200000,0.200000,0.200000
400,No log,1.635938,0.200000,0.200000,0.200000,0.200000
500,1.651100,1.615756,0.200000,0.200000,0.200000,0.200000
600,1.651100,1.624851,0.200000,0.200000,0.200000,0.200000
700,1.651100,1.612169,0.200000,0.200000,0.200000,0.200000


<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>


In [26]:
train_metrics = trainer.state.log_history
train_metrics
# # Extract relevant information
# train_loss = [metric['train_loss'] for metric in train_metrics]  # Training loss
# train_acc = [metric['eval_accuracy'] for metric in train_metrics]  # Training accuracy, if available

[{'eval_loss': 1.671751618385315,
  'eval_accuracy': 0.2,
  'eval_precision': 0.2,
  'eval_recall': 0.2,
  'eval_f1': 0.20000000000000004,
  'eval_runtime': 14.4065,
  'eval_samples_per_second': 34.707,
  'eval_steps_per_second': 4.373,
  'epoch': 0.4,
  'step': 100},
 {'eval_loss': 1.633170485496521,
  'eval_accuracy': 0.2,
  'eval_precision': 0.2,
  'eval_recall': 0.2,
  'eval_f1': 0.20000000000000004,
  'eval_runtime': 14.8459,
  'eval_samples_per_second': 33.679,
  'eval_steps_per_second': 4.244,
  'epoch': 0.8,
  'step': 200},
 {'eval_loss': 1.633176565170288,
  'eval_accuracy': 0.2,
  'eval_precision': 0.2,
  'eval_recall': 0.2,
  'eval_f1': 0.20000000000000004,
  'eval_runtime': 15.0402,
  'eval_samples_per_second': 33.244,
  'eval_steps_per_second': 4.189,
  'epoch': 1.2,
  'step': 300},
 {'eval_loss': 1.6359384059906006,
  'eval_accuracy': 0.2,
  'eval_precision': 0.2,
  'eval_recall': 0.2,
  'eval_f1': 0.20000000000000004,
  'eval_runtime': 15.6419,
  'eval_samples_per_second

In [29]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 1.6109877824783325,
 'eval_accuracy': 0.2,
 'eval_precision': 0.2,
 'eval_recall': 0.2,
 'eval_f1': 0.20000000000000004,
 'eval_runtime': 15.0148,
 'eval_samples_per_second': 33.301,
 'eval_steps_per_second': 4.196,
 'epoch': 3.0}

In [30]:
evaluate_metrics = trainer.state.log_history
evaluate_metrics

[{'eval_loss': 1.671751618385315,
  'eval_accuracy': 0.2,
  'eval_precision': 0.2,
  'eval_recall': 0.2,
  'eval_f1': 0.20000000000000004,
  'eval_runtime': 14.4065,
  'eval_samples_per_second': 34.707,
  'eval_steps_per_second': 4.373,
  'epoch': 0.4,
  'step': 100},
 {'eval_loss': 1.633170485496521,
  'eval_accuracy': 0.2,
  'eval_precision': 0.2,
  'eval_recall': 0.2,
  'eval_f1': 0.20000000000000004,
  'eval_runtime': 14.8459,
  'eval_samples_per_second': 33.679,
  'eval_steps_per_second': 4.244,
  'epoch': 0.8,
  'step': 200},
 {'eval_loss': 1.633176565170288,
  'eval_accuracy': 0.2,
  'eval_precision': 0.2,
  'eval_recall': 0.2,
  'eval_f1': 0.20000000000000004,
  'eval_runtime': 15.0402,
  'eval_samples_per_second': 33.244,
  'eval_steps_per_second': 4.189,
  'epoch': 1.2,
  'step': 300},
 {'eval_loss': 1.6359384059906006,
  'eval_accuracy': 0.2,
  'eval_precision': 0.2,
  'eval_recall': 0.2,
  'eval_f1': 0.20000000000000004,
  'eval_runtime': 15.6419,
  'eval_samples_per_second

In [31]:
text = """Decent poke at an okay price. We got two bowls with sushi rice: salmon and albacore, and salmon and seared cajun tuna.
Good: The salmon and tuna were good, and there's a large selection of  mixins and addons -- I consistently get the crab salad, seaweed salad, and tofu skins.

Bad: The albacore was overly salty and many of the pieces were overcooked and rubbery. The sauces are good but I wish there was more added, without having to pay extra.

Overall, nothing spectacular, but Pokeman is an average option in University City."""
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[0.5241, 0.5775, 0.4039, 0.5131, 0.5068]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.2035, 0.2147, 0.1805, 0.2013, 0.2000]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)


array([[0.2035177 , 0.2146787 , 0.18047269, 0.20129254, 0.20003828]],
      dtype=float32)

In [32]:
print("Ratings :" ,float(np.argmax(predictions) + 1 ))

Ratings : 2.0


In [34]:
torch.save(model, 'path/to/model') #to save

saved_model = torch.load('path/to/model') # to load